# ECE367: PS02 Part 2 -- PageRank

## Framing

$N$ is the number of URL's. $J$ is the adjacency matrix.

### Power Iteration Method

* Method for calculating eigenvalues/vectors for diagonalizable matrix. 
* 

## Steps

- [x] Load the pagerank data from `pagerank_urls.txt`, `pagerank_adj.mat`.
- [x] Based on adjacency matrix $J$ calculate $$A_{i, j} = \frac{J_{i, j}}{\sum_{k=1}^{N}J_{k,j}}$$
    - [x] Verify that the rows add to 1.
- [x] Implement the **power iteration method** (OptM 7.1.1) for 10 iterations.
    - [x] Calculate $e(k+1) = ||Ax(k+1) - x(k+1)||_2$
    - [x] Plot $\log(e(k+1))$ vs. k
- [ ] Implement **shift-invert power iteration** and **Rayleigh quotient iteration** algorithms (OptM 7.1.2, 7.1.3). 
    - [ ] For shift-invert: $\sigma = 0.99$.
    - [ ] For Rayleigh quotient: $\sigma_1 = \sigma_2 = 0.99$ for first two iterations.
        - [ ] $\sigma_k = \frac{x^*(k) Ax(k)}{x^*(k)x(k)}$ for k > 2.
    - [ ] Plot $\log(e(k+1))$ vs. k for each plot.
- [ ] List the (page index, PageRank score) tuples for **top 5** and **bottom 5** pages according to PageRank scores.

In [2]:
# IMPORT BOX #
# IMPORT BOX #
using Plots
using Plotly
using GR
using SymPy
using MAT
using LinearAlgebra

plotly()

Plots.PlotlyBackend()

In [26]:
file_J = matopen("../PS02_dataSet/pagerank_adj.mat")
J = read(file_J)["J"];
typeof(J)

Array{Float64,2}

In [27]:
# Normalization function for J
function normalize_J(J::Array{Float64,2})
    A = zeros(size(J))
    
    for i = 1:size(J,1)
        if(sum(J[:,i]) == 0)
            println("Sum of row ",i," equals 0...")
        else
            A[:,i] = J[:,i]/sum(J[:,i])
        end
        
    end
    A
end

normalize_J (generic function with 1 method)

In [28]:
A = normalize_J(J);
x = sum(A, dims=1);

In [29]:
good_sum = true

for i = 1:size(x,1)
    if abs(x[i] - 1) > 0.00001
        good_sum = false
        println("incorrect sum at ",i," -- x[i] = ",x[i])
    end
end

if good_sum
    println("Sum of A's columns are all 1. Checks out")
else
    println("Sum of A's columns are not all 1.")
end

x = transpose(x); # Turning x into a column vector

Sum of A's columns are all 1. Checks out


In [30]:
function get_err(A::Array{Float64,2}, x)
    norm(A*x - x)
end

get_err (generic function with 1 method)

In [31]:
function power_iteration(A::Array{Float64,2}, xin, num_iters)
    x = copy(xin)
    if norm(x) != 1
        x = x./norm(x)
    end
    
    errors = zeros(num_iters)
    
    λ = 0   
    for k = 1:num_iters
        y = A*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end
    
    return (x, errors)
end

power_iteration (generic function with 1 method)

In [32]:
function shift_invert_power_iteration(A::Array{Float64,2}, xin, num_iters, σ=0.99)
    x = copy(xin)
    if norm(x) != 1
        x = x./norm(x)
    end
    
    errors = zeros(num_iters)
    
    λ = 0   
    n = size(A,1)
    for k = 1:num_iters
        y = inv(A-(σ.*I(n)))*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end
    
    return (x, errors)
end

shift_invert_power_iteration (generic function with 2 methods)

In [33]:
function rayleigh_quotient_iteration(A::Array{Float64,2}, xin, num_iters, σ=0.99)
    x = copy(xin)
    
    if norm(x) != 1
        x = x./norm(x)
    end

    errors = zeros(num_iters)
    λ = 0
    n = size(A,1)
    for k = 1:num_iters
        if k > 2
            σ = (transpose(x)*A*x)/(transpose(x)*x)
        end
        y = inv(A-(σ.*I(n)))*x
        x = y/norm(y)
        λ = transpose(x)*A*x
        errors[k] = get_err(A, x)
    end

    return (x, errors)
end

rayleigh_quotient_iteration (generic function with 2 methods)

In [34]:
x_pi, errors_pi = power_iteration(A, x, 10);
x_sipi, errors_sipi = shift_invert_power_iteration(A, x, 10);
x_rqi, errors_rqi = rayleigh_quotient_iteration(A, x, 10);

In [47]:
Plots.plot(log10.(errors_pi), label="e(k+1)")
Plots.title!("Power Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("log e(k+1)")
# Plots.savefig("figures/2_9_PI_err_it.png")

<!DOCTYPE html>
 
 
 Plots.jl

In [48]:
Plots.plot(log10.(errors_sipi), label="e(k+1)")
Plots.title!("Shift-Invert Power Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("e(k+1)")
# Plots.savefig("figures/2_9_SIPI_err_it.png")

<!DOCTYPE html>
 
 
 Plots.jl

In [49]:
Plots.plot(log10.(errors_rqi), label="e(k+1)")
Plots.title!("Rayleigh Quotient Iteration: Error vs. Iteration")
Plots.xlabel!("k")
Plots.ylabel!("log e(k+1)")
# Plots.savefig("figures/2_9_RQI_err_it.png")

<!DOCTYPE html>
 
 
 Plots.jl

In [50]:
# Using the text file, we report the most 'important' pages
namefile = open("PS02_dataSet/pagerank_urls.txt");
noms = readlines(namefile);

LoadError: SystemError: opening file "PS02_dataSet/pagerank_urls.txt": No such file or directory

In [51]:
function get_top_n(x, n, absolute=false)
    x_cpy = copy(x)
    
    if absolute
        x_cpy = abs.(x_cpy)
    end
    
    indexes = zeros(Int16, n)
    
    for i = 1:n
        mxval, mxindx = findmax(x_cpy)
        idx = mxindx[1]
        indexes[i] = convert(Int16, idx)
        
        x_cpy[idx] = 0
    end
    
    indexes
end
function get_bottom_n(x, n, absolute=false)
    x_cpy = copy(x)
    
    if absolute
        x_cpy = abs.(x_cpy)
    end
    
    indexes = zeros(Int16, n)
    
    for i = 1:n
        mxval, mxindx = findmin(x_cpy)
        idx = mxindx[1]
        indexes[i] = convert(Int16, idx)
        
        x_cpy[idx] = 1
    end
    
    indexes
end

get_bottom_n (generic function with 2 methods)

In [52]:
idxs = get_top_n(x_pi, 5)
println("For Power Iteration")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_pi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_pi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_pi[idxs[i]])
end

For Power Iteration


LoadError: UndefVarError: noms not defined

In [53]:
idxs = get_top_n(x_sipi, 5)
println("For Shift-Invert Power Iteration (Not Taking Absolute Values)")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_sipi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end

idxs = get_top_n(x_sipi, 5, true)
println("\n\n\nFor Shift-Invert Power Iteration (Absolute Values)")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_sipi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ",x_sipi[idxs[i]])
end

For Shift-Invert Power Iteration (Not Taking Absolute Values)


LoadError: UndefVarError: noms not defined

In [54]:
idxs = get_top_n(x_rqi, 5)
println("For Rayleigh Quotient Iteration")
for i = 1:5
    println("Top ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ", x_rqi[idxs[i]])
end
println("--")
idxs = get_bottom_n(x_rqi, 5)
for i = 1:5
    println("Bottom ",i," Page: ",noms[idxs[i]]," at index ",idxs[i]," with score ", x_rqi[idxs[i]])
end

For Rayleigh Quotient Iteration


LoadError: UndefVarError: noms not defined

In [55]:
get_err(A, x_pi)

0.11297845011565136

In [56]:
get_err(A, x_sipi)

0.000783822615251815

In [57]:
get_err(A, x_rqi)

1.6786171296150374e-16

In [60]:
Plots.plot(x_pi, label="Power Iteration")
Plots.plot!(x_sipi, label="Shift-Inverte Power Method")
Plots.plot!(x_rqi, label="Rayleigh")
Plots.title!("Line Graphs EigVecs")
# Plots.savefig("guhguh")

<!DOCTYPE html>
 
 
 Plots.jl